# Musical_Instruments_5.json

* Read dataset
* Preprocess data
* Use Product ID , reviewerID and overall > build model to predict overalls > recommend to users

In [1]:
import pyspark
from pyspark import SparkContext

sc = SparkContext(master='local', appName = 'Recommendation_System')

In [2]:
import findspark
findspark.init()
import pyspark
findspark.find()

'/Users/vunguyen/server/spark-2.4.6-bin-hadoop2.7'

In [3]:
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
'''
conf= pyspark.SparkConf().setAppName('tets_app').setMaster('local')
sc=pyspark.SparkContext(conf=conf)
sc
'''

#sc= SparkContext()
spark = (SparkSession
         .builder
         .appName('Recommendation_System')
         .getOrCreate())
spark

In [5]:
data = spark.read.json('File/Musical_Instruments_5.json')

In [6]:
data.show(5, truncate=True)

+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|      asin| helpful|overall|          reviewText| reviewTime|    reviewerID|        reviewerName|             summary|unixReviewTime|
+----------+--------+-------+--------------------+-----------+--------------+--------------------+--------------------+--------------+
|1384719342|  [0, 0]|    5.0|Not much to write...|02 28, 2014|A2IBPI20UZIR0U|cassandra tu "Yea...|                good|    1393545600|
|1384719342|[13, 14]|    5.0|The product does ...|03 16, 2013|A14VAT5EAX3D9S|                Jake|                Jake|    1363392000|
|1384719342|  [1, 1]|    5.0|The primary job o...|08 28, 2013|A195EZSQDW3E21|Rick Bennette "Ri...|It Does The Job Well|    1377648000|
|1384719342|  [0, 0]|    5.0|Nice windscreen p...|02 14, 2014|A2C00NNG1ZQQG2|RustyBill "Sunday...|GOOD WINDSCREEN F...|    1392336000|
|1384719342|  [0, 0]|    5.0|This pop filter i...|02 21

In [7]:
data_sub = data.select(['asin','overall','reviewerID'])

In [8]:
data_sub.count()

10261

In [9]:
from pyspark.sql.functions import *

In [11]:
data_sub.show(5, truncate=True)

+----------+-------+--------------+
|      asin|overall|    reviewerID|
+----------+-------+--------------+
|1384719342|    5.0|A2IBPI20UZIR0U|
|1384719342|    5.0|A14VAT5EAX3D9S|
|1384719342|    5.0|A195EZSQDW3E21|
|1384719342|    5.0|A2C00NNG1ZQQG2|
|1384719342|    5.0| A94QU4C90B1AX|
+----------+-------+--------------+
only showing top 5 rows



In [12]:
data_sub.select([count(when(col(c).isNull(),c)).alias(c) for c in data_sub.columns]).toPandas().T

,0
asin,0
overall,0
reviewerID,0


In [13]:
users = data_sub.select('reviewerID').distinct().count()
products = data_sub.select('asin').distinct().count()
numerator = data_sub.count()

In [14]:
display(numerator,users,products)

10261

1429

900

# Density = 900/1429 ~ 60%

In [15]:
denom = users * products
sparsity = 1 - (numerator*1.0/ denom)
print('Sparsity: ', sparsity)

Sparsity:  0.992021615737501


In [16]:
from pyspark.ml.evaluation import RegressionEvaluator
from pyspark.ml.recommendation import ALS

# Transform asin, reviewerID

In [17]:
from pyspark.ml.feature import StringIndexer
from pyspark.ml import Pipeline
from pyspark.sql.functions import col


In [18]:
indexer = StringIndexer(inputCol = 'asin', outputCol='asin_idx')
indexer_model = indexer.fit(data_sub)
data_indexed = indexer_model.transform(data_sub)

In [19]:
indexer1 = StringIndexer(inputCol = 'reviewerID', outputCol='reviewerID_idx')
indexer1_model = indexer1.fit(data_sub)
data_indexed = indexer1_model.transform(data_indexed)

In [20]:
data_indexed.show(5, truncate=True)

+----------+-------+--------------+--------+--------------+
|      asin|overall|    reviewerID|asin_idx|reviewerID_idx|
+----------+-------+--------------+--------+--------------+
|1384719342|    5.0|A2IBPI20UZIR0U|   781.0|          72.0|
|1384719342|    5.0|A14VAT5EAX3D9S|   781.0|         359.0|
|1384719342|    5.0|A195EZSQDW3E21|   781.0|         436.0|
|1384719342|    5.0|A2C00NNG1ZQQG2|   781.0|        1216.0|
|1384719342|    5.0| A94QU4C90B1AX|   781.0|        1137.0|
+----------+-------+--------------+--------+--------------+
only showing top 5 rows



In [22]:
data_indexed.select([count(when(col(c).isNull(),c)).alias(c) for c in data_indexed.columns]).toPandas().T

,0
asin,0
overall,0
reviewerID,0
asin_idx,0
reviewerID_idx,0


In [23]:
(train_data,test_data) = data_indexed.randomSplit([0.8,0.2])

In [25]:
als = ALS(maxIter = 5, regParam=0.09, rank = 25,
         userCol='reviewerID_idx',
         itemCol='asin_idx',
         ratingCol='overall',
         coldStartStrategy='drop',
         nonnegative=True)
model = als.fit(train_data)

In [26]:
predictions = model.transform(test_data)

In [27]:
predictions.select(['asin_idx','reviewerID_idx','overall','prediction']).show(5)

+--------+--------------+-------+----------+
|asin_idx|reviewerID_idx|overall|prediction|
+--------+--------------+-------+----------+
|   148.0|         356.0|    4.0| 3.6249642|
|   148.0|         344.0|    3.0| 4.2484736|
|   148.0|          30.0|    3.0|  4.871785|
|   463.0|        1165.0|    4.0|  3.672851|
|   463.0|          27.0|    5.0| 4.9877906|
+--------+--------------+-------+----------+
only showing top 5 rows



In [28]:
evaluator = RegressionEvaluator(metricName='rmse',
                               labelCol='overall',
                               predictionCol='prediction')

rmse = evaluator.evaluate(predictions)
print('RMSE = ' +str(rmse))

RMSE = 1.211244139043411


In [29]:
user_recs = model.recommendForAllUsers(20)

In [30]:
for user in user_recs.head(5):
    print(user)
    print("\n")

Row(reviewerID_idx=471, recommendations=[Row(asin_idx=621, rating=6.0567731857299805), Row(asin_idx=888, rating=5.854616641998291), Row(asin_idx=738, rating=5.773068428039551), Row(asin_idx=724, rating=5.771135330200195), Row(asin_idx=853, rating=5.769525527954102), Row(asin_idx=805, rating=5.763358116149902), Row(asin_idx=377, rating=5.752736568450928), Row(asin_idx=544, rating=5.734504699707031), Row(asin_idx=766, rating=5.686485767364502), Row(asin_idx=482, rating=5.6559157371521), Row(asin_idx=779, rating=5.634638786315918), Row(asin_idx=547, rating=5.63284158706665), Row(asin_idx=426, rating=5.629472732543945), Row(asin_idx=328, rating=5.616199493408203), Row(asin_idx=616, rating=5.596076965332031), Row(asin_idx=181, rating=5.551751613616943), Row(asin_idx=548, rating=5.542877197265625), Row(asin_idx=595, rating=5.538969993591309), Row(asin_idx=630, rating=5.534778118133545), Row(asin_idx=582, rating=5.508908271789551)])


Row(reviewerID_idx=1342, recommendations=[Row(asin_idx=621

# Show the reviewer ID

In [31]:
import pandas as pd

recs = model.recommendForAllUsers(10).toPandas()
nrecs = recs.recommendations.apply(pd.Series).merge(recs, right_index = True, left_index = True).drop(['recommendations'], axis=1).melt(id_vars=['reviewerID_idx'], value_name='recommendations').drop('variable', axis=1).dropna()
nrecs = nrecs.sort_values('reviewerID_idx')
nrecs= pd.concat([nrecs['recommendations'].apply(pd.Series),
                 nrecs['reviewerID_idx']], axis=1)
nrecs.columns = ['ProductID_index', 'Rating', 'UserID_index']

In [36]:
md = data_indexed.select(['reviewerID','reviewerID_idx','asin','asin_idx'])
md = md.toPandas()
dict1=dict(zip(md['reviewerID_idx'],md['reviewerID']))
dict2=dict(zip(md['asin_idx'],md['asin']))
nrecs['reviewerID']=nrecs['UserID_index'].map(dict1)
nrecs['asin']=nrecs['ProductID_index'].map(dict2)
nrecs=nrecs.sort_values('reviewerID')
nrecs.reset_index(drop=True, inplace=True)
new = nrecs[['reviewerID','asin','Rating']]
new['recommendations'] = list(zip(new.asin, new.Rating))
res=new[['reviewerID','recommendations']]
res_new = res['recommendations'].groupby([res.reviewerID]).apply(list).reset_index()

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.


In [37]:
res_new

,reviewerID,recommendations
0,A00625243BI8W1SSZNLMD,"[(B003FW0IHA, 7.1307268142700195), (B004PFWZHM..."
1,A10044ECXDUVKS,"[(B005EHILV4, 5.522689342498779), (B001E3SFKO,..."
2,A102MU6ZC9H1N6,"[(B003FW0IHA, 5.881282806396484), (B0002ZO60I,..."
3,A109JTUZXO61UY,"[(B002PAW6EG, 5.745532989501953), (B003FW0IHA,..."
4,A109ME7C09HM2M,"[(B003AYNFRS, 6.1417622566223145), (B0038MTH8S..."
...,...,...
1424,AZJPNK73JF3XP,"[(B000BW2D84, 5.486822128295898), (B003FW0IHA,..."
1425,AZMHABTPXVLG3,"[(B0013V1BYY, 3.9393177032470703), (B002T45X1G..."
1426,AZMIKIG4BB6BZ,"[(B0002CZVHI, 5.503294467926025), (B001L8IJ0I,..."
1427,AZPDO6FLSMLFP,"[(B0006H92QK, 4.945075035095215), (B0002GIRP2,..."
